In [1]:
import pandas as pd

In [6]:
df_dh = pd.read_csv('../data/horsey.csv')
df_rc = pd.read_csv('../data/rc.csv')

In [7]:
print(df_dh.iloc[0])

round_date                                   2016-11-18
round_no                                              2
hole_no                                              13
shot_no                                               1
putt                                                NaN
completed                                          True
left_to_pin_before                                 7560
left_to_pin_after                                   360
penalty                                             NaN
player_name                                David Horsey
name                  DP World Tour Championship, Dubai
course_id                                            48
start_tee                                             1
score                                                 3
par                                                   3
yards                                               204
lie_after                                         Green
lie_before                                      

In [8]:
print(df_rc.iloc[0])

left_to_pin_before                         144
penalty                                  False
hole_no                                     14
shot_no                                      3
completed                                 True
putt                                      True
left_to_pin_after                            1
par                                          4
yards                                      448
mp_name               Friday Foursomes Match 4
usa_player1                        Matt Kuchar
usa_player2                     Dustin Johnson
eur_player1                       Lee Westwood
eur_player2                     Thomas Pieters
mp_start_time              2016-09-30 13:20:00
player_shot_name                   Matt Kuchar
shot_team                                  USA
lie_after                                 Hole
lie_before                               Green
Name: 0, dtype: object


In [9]:
df_dh.head()

,round_date,round_no,hole_no,shot_no,putt,completed,left_to_pin_before,left_to_pin_after,penalty,player_name,name,course_id,start_tee,score,par,yards,lie_after,lie_before,club,direction
0,2016-11-18,2,13,1,NaN,True,7560,360.0,NaN,David Horsey,"DP World Tour Championship, Dubai",48,1,3,3,204,Green,Tee,iron,center
1,2016-11-18,2,13,2,True,True,360,12.0,NaN,David Horsey,"DP World Tour Championship, Dubai",48,1,3,3,204,Green,Green,putter,short
2,2016-11-18,2,13,3,True,True,12,1.0,NaN,David Horsey,"DP World Tour Championship, Dubai",48,1,3,3,204,Hole,Green,putter,center
3,2016-11-18,2,18,1,NaN,True,19440,9324.0,NaN,David Horsey,"DP World Tour Championship, Dubai",48,1,4,5,620,Fairway,Tee,driver,center
4,2016-11-18,2,18,2,NaN,True,9324,2520.0,NaN,David Horsey,"DP World Tour Championship, Dubai",48,1,4,5,620,Fairway,Fairway,iron,center
